# [python3 cookbook](https://python3-cookbook.readthedocs.io/zh_CN/latest/preface.html)

## 第一章：数据结构和算法
### 解压序列赋值给多个变量

**问题**

现在有一个包含 N 个元素的元组或者是序列，怎样将它里面的值解压后同时赋值给 N 个变量？

**解决方案**

任何的序列（或者是可迭代对象）可以通过一个简单的赋值语句解压并赋值给多个变量。 唯一的前提就是变量的数量必须跟序列元素的数量是一样的。

代码示例：

In [4]:
p = (4,5)
x,y = p
print('x = ',x)
print('y = ',y)

data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
name, shares, price, date = data
print('name:{}, shares:{}, price:{}, date:{}'.format(name, shares, price, date))

name, shares, price, (year, mon, day) = data
print('name:{}, shares:{}, price:{}, date:{}year,{}mon,{}day'.format(name, shares, price, year, mon, day))

x =  4
y =  5
name:ACME, shares:50, price:91.1, date:(2012, 12, 21)
name:ACME, shares:50, price:91.1, date:2012year,12mon,21day


如果变量个数和序列元素的个数不匹配，会产生一个异常。

In [5]:
p = (4,5)
x,y,z = p

ValueError: not enough values to unpack (expected 3, got 2)

**讨论**

实际上，这种解压赋值可以用在任何可迭代对象上面，而不仅仅是列表或者元组。 包括字符串，文件对象，迭代器和生成器。

代码示例：

In [6]:
s = 'Hello'
a, b, c, d, e = s
print(a,b,c,d,e)

H e l l o


有时候，你可能只想解压一部分，丢弃其他的值。对于这种情况 Python 并没有提供特殊的语法。 但是你可以使用任意变量名去占位，到时候丢掉这些变量就行了。你必须保证你选用的那些占位变量名在其他地方没被使用到。

代码示例：

In [7]:
data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
_, shares, price, _ = data
print(shares,price)

50 91.1


### 解压可迭代对象赋值给多个变量

**问题**

如果一个可迭代对象的元素个数超过变量个数时，会抛出一个 ValueError 。 那么怎样才能从这个可迭代对象中解压出 N 个元素出来？

**解决方案**

Python 的星号表达式可以用来解决这个问题。比如，你在学习一门课程，在学期末的时候， 你想统计下家庭作业的平均成绩，但是排除掉第一个和最后一个分数。如果只有四个分数，你可能就直接去简单的手动赋值， 但如果有 24 个呢？这时候星号表达式就派上用场了：

In [10]:
def drop_first_last(grades):
    first,*middle,last = grades
    return sum(middle)/len(middle)

grades = [24,57,99,56,33,74,99]
drop_first_last(grades)

63.8

另外一种情况，假设你现在有一些用户的记录列表，每条记录包含一个名字、邮件，接着就是不确定数量的电话号码。 你可以像下面这样分解这些记录：

In [11]:
record = ('Dave', 'dave@example.com', '773-555-1212', '847-555-1212')
name, email, *phone_numbers = record
print(name, email, phone_numbers)

Dave dave@example.com ['773-555-1212', '847-555-1212']


值得注意的是上面解压出的 phone_numbers 变量永远都是列表类型，不管解压的电话号码数量是多少（包括 0 个）。 所以，任何使用到 phone_numbers 变量的代码就不需要做多余的类型检查去确认它是否是列表类型了。

星号表达式也能用在列表的开始部分。比如，你有一个公司前 8 个月销售数据的序列， 但是你想看下最近一个月数据和前面 7 个月的平均值的对比。你可以这样做：

```
*trailing_qtrs, current_qtr = sales_record
trailing_avg = sum(trailing_qtrs) / len(trailing_qtrs)
return avg_comparison(trailing_avg, current_qtr)
```

In [12]:
*trailing, current = [10, 8, 7, 1, 9, 5, 10, 3]
print(trailing,current)

[10, 8, 7, 1, 9, 5, 10] 3


**讨论**

扩展的迭代解压语法是专门为解压不确定个数或任意个数元素的可迭代对象而设计的。 通常，这些可迭代对象的元素结构有确定的规则（比如第 1 个元素后面都是电话号码）， 星号表达式让开发人员可以很容易的利用这些规则来解压出元素来。 而不是通过一些比较复杂的手段去获取这些关联的元素值。

值得注意的是，**星号表达式在迭代元素为可变长元组的序列时是很有用的**。 比如，下面是一个带有标签的元组序列：

In [14]:
records = [('foo', 1, 2),('bar', 'hello'),('foo', 3, 4)]

def do_foo(x,y):
    print('foo',x,y)
    
def do_bar(s):
    print('bar',s)
    
for tag,*args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


**星号解压语法在字符串操作的时候也会很有用**，比如字符串的分割。

In [15]:
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *fields, homedir, sh = line.split(':')
print(uname, fields, homedir, sh)

nobody ['*', '-2', '-2', 'Unprivileged User'] /var/empty /usr/bin/false


有时候，你想解压一些元素后丢弃它们，你不能简单就使用 * ， 但是你可以使用一个普通的废弃名称，比如 _ 或者 ign （ignore）。

In [16]:
record = ('ACME', 50, 123.45, (12, 18, 2012))
name, *_, (*_, year) = record
print(name,year)

ACME 2012


在很多函数式语言中，星号解压语法跟列表处理有许多相似之处。比如，如果你有一个列表， 你可以很容易的将它分割成前后两部分：

In [17]:
items = [1, 10, 7, 4, 5, 9]
head, *tail = items
print(head,tail)

1 [10, 7, 4, 5, 9]


如果你够聪明的话，还能用这种分割语法去巧妙的实现递归算法。比如：

In [18]:
def sum(items):
    head,*tail = items
    return head+sum(tail) if tail else head

items = [1, 10, 7, 4, 5, 9]
sum(items)

36

列表推导式 if-else
```
if tail:
    return head + sum(tail)
else:
    return head
```
然后，由于语言层面的限制，递归并不是 Python 擅长的。 因此，最后那个递归演示仅仅是个好奇的探索罢了，对这个不要太认真了。

### 保留最后 N 个元素

**问题**
在迭代操作或者其他操作的时候，怎样只保留最后有限几个元素的历史记录？

**解决方案**
保留有限历史记录正是 collections.deque 大显身手的时候。比如，下面的代码在多行上面做简单的文本匹配， 并返回匹配所在行的最后N行：

In [22]:
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            yield line, previous_lines # 
        previous_lines.append(line)

# Example use on a file
if __name__ == '__main__':
    with open(r'cookbook_file_1.3.txt') as f:
        for line, prevlines in search(f, 'python', 5):
            for pline in prevlines:
                print(pline, end='')
            print(line, end='')
            print('-' * 20)

this is line one.
this is line two.(python)
--------------------
this is line one.
this is line two.(python)
this is line three.
this is line four.(python)
--------------------
this is line one.
this is line two.(python)
this is line three.
this is line four.(python)
this is line five.
this is line six.(python)
--------------------
this is line three.
this is line four.(python)
this is line five.
this is line six.(python)
this is line seven.
this is line eight.(python)
--------------------
this is line five.
this is line six.(python)
this is line seven.
this is line eight.(python)
this is line nine.
this is line ten.(python)--------------------


**讨论**

我们在写查询元素的代码时，通常会使用包含 yield 表达式的生成器函数，也就是我们上面示例代码中的那样。 这样可以将搜索过程代码和使用搜索结果代码解耦。如果你还不清楚什么是生成器，请参看 4.3 节。

使用 deque(maxlen=N) 构造函数会新建一个固定大小的队列。当新的元素加入并且这个队列已满的时候， 最老的元素会自动被移除掉。

In [5]:
q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
print(q)
q.append(4)
print(q)

deque([1, 2, 3], maxlen=3)
deque([2, 3, 4], maxlen=3)


尽管你也可以手动在一个列表上实现这一的操作（比如增加、删除等等）。但是这里的队列方案会更加优雅并且运行得更快些。

更一般的， deque 类可以被用在任何你只需要一个简单队列数据结构的场合。 如果你不设置最大队列大小，那么就会得到一个无限大小队列，你可以在队列的两端执行添加和弹出元素的操作。

在队列两端插入或删除元素时间复杂度都是 O(1) ，区别于列表，在列表的开头插入或删除元素的时间复杂度为 O(N) 。

In [7]:
q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
print(q)
q.appendleft(4)
print(q)
q.pop()
print(q)
q.popleft()
print(q)

deque([1, 2, 3], maxlen=3)
deque([4, 1, 2], maxlen=3)
deque([4, 1], maxlen=3)
deque([1], maxlen=3)


### 查找最大或最小的 N 个元素

**问题**

怎样从一个集合中获得最大或者最小的 N 个元素列表？

**解决方案**

heapq 模块有两个函数：nlargest() 和 nsmallest() 可以完美解决这个问题。

In [8]:
import heapq
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3,nums))
print(heapq.nsmallest(3,nums))

[42, 37, 23]
[-4, 1, 2]


两个函数都能接受一个关键字参数，用于更复杂的数据结构中：

In [13]:
portfolio = [
    {'name': 'IBM', 'shares': 100, 'price': 91.1},
    {'name': 'AAPL', 'shares': 50, 'price': 543.22},
    {'name': 'FB', 'shares': 200, 'price': 21.09},
    {'name': 'HPQ', 'shares': 35, 'price': 31.75},
    {'name': 'YHOO', 'shares': 45, 'price': 16.35},
    {'name': 'ACME', 'shares': 75, 'price': 115.65}
]

cheap = heapq.nsmallest(3,portfolio,key=lambda s:s['price'])
expensive = heapq.nlargest(3,portfolio,key=lambda s:s['price'])
print(cheap)
print(expensive)

[{'name': 'YHOO', 'shares': 45, 'price': 16.35}, {'name': 'FB', 'shares': 200, 'price': 21.09}, {'name': 'HPQ', 'shares': 35, 'price': 31.75}]
[{'name': 'AAPL', 'shares': 50, 'price': 543.22}, {'name': 'ACME', 'shares': 75, 'price': 115.65}, {'name': 'IBM', 'shares': 100, 'price': 91.1}]


上面代码在对每个元素进行对比的时候，会以 price 的值进行比较。

**讨论**

如果你想在一个集合中查找最小或最大的 N 个元素，并且 N 小于集合元素数量，那么这些函数提供了很好的性能。 因为在底层实现里面，首先会先将集合数据进行堆排序后放入一个列表中：

In [19]:
import heapq
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
heap = list(nums)
heapq.heapify(heap)
print(heap)

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]


堆数据结构最重要的特征是 heap[0] 永远是最小的元素。并且剩余的元素可以很容易的通过调用 heapq.heappop() 方法得到， 该方法会先将第一个元素弹出来，然后用下一个最小的元素来取代被弹出元素（这种操作时间复杂度仅仅是 O(log N)，N 是堆大小）。 比如，如果想要查找最小的 3 个元素，你可以这样做：

In [20]:
print(heapq.heappop(heap))
print(heapq.heappop(heap))
print(heapq.heappop(heap))

-4
1
2


当要查找的元素个数相对比较小的时候，函数 nlargest() 和 nsmallest() 是很合适的。 如果你仅仅想查找唯一的最小或最大（N=1）的元素的话，那么使用 min() 和 max() 函数会更快些。 类似的，如果 N 的大小和集合大小接近的时候，通常先排序这个集合然后再使用切片操作会更快点 （ sorted(items)[:N] 或者是 sorted(items)[-N:] ）。 需要在正确场合使用函数 nlargest() 和 nsmallest() 才能发挥它们的优势 （如果 N 快接近集合大小了，那么使用排序操作会更好些）。

尽管你没有必要一定使用这里的方法，但是堆数据结构的实现是一个很有趣并且值得你深入学习的东西。 基本上只要是数据结构和算法书籍里面都会有提及到。 heapq 模块的官方文档里面也详细的介绍了堆数据结构底层的实现细节。

### 实现一个优先级队列

**问题**

怎样实现一个按优先级排序的队列？ 并且在这个队列上面每次 pop 操作总是返回优先级最高的那个元素

**解决方案**

下面的类利用 heapq 模块实现了一个简单的优先级队列：

In [30]:
import heapq

class PriorityQueue():
    def __init__(self):
        self._queue = []
        self._index = 0
        
    def push(self,item,priority):
        heapq.heappush(self._queue,(-priority,self._index,item))
        self._index += 1
        
    def pop(self):
        return heapq.heappop(self._queue)[-1]
    
class Item:
    def __init__(self,name):
        self.name = name
    def __repr__(self):
        return 'Item({!r})'.format(self.name)
    
q = PriorityQueue()
q.push(Item('foo'), 1)
q.push(Item('bar'), 5)
q.push(Item('spam'), 4)
q.push(Item('grok'), 1)
for i in range(4):
    print(q.pop())

Item('bar')
Item('spam')
Item('foo')
Item('grok')


仔细观察可以发现，第一个 pop() 操作返回优先级最高的元素。 另外注意到如果两个有着相同优先级的元素（ foo 和 grok ），pop 操作按照它们被插入到队列的顺序返回的。

**讨论**

这一小节我们主要关注 heapq 模块的使用。 函数 heapq.heappush() 和 heapq.heappop() 分别在队列 _queue 上插入和删除第一个元素， 并且队列 _queue 保证第一个元素拥有最高优先级（ 1.4 节已经讨论过这个问题）。 heappop() 函数总是返回”最小的”的元素，这就是保证队列pop操作返回正确元素的关键。 另外，由于 push 和 pop 操作时间复杂度为 O(log N)，其中 N 是堆的大小，因此就算是 N 很大的时候它们运行速度也依旧很快。

在上面代码中，队列包含了一个 (-priority, index, item) 的元组。 优先级为负数的目的是使得元素按照优先级从高到低排序。 这个跟普通的按优先级从低到高排序的堆排序恰巧相反。

index 变量的作用是保证同等优先级元素的正确排序。 通过保存一个不断增加的 index 下标变量，可以确保元素按照它们插入的顺序排序。 而且， index 变量也在相同优先级元素比较的时候起到重要作用。

In [27]:
from heapq import heappush,heappop

h = []
heappush(h, (5, 1,'write code'))
heappush(h, (7, 2,'release product'))
heappush(h, (1, 3,'write spec'))
heappush(h, (3, 4,'create tests'))
print(heappop(h))
print(heappop(h)[-1])

(1, 3, 'write spec')
create tests


### 字典中的键映射多个值

**问题**

怎样实现一个键对应多个值的字典（也叫 multidict）？

**解决方案**

一个字典就是一个键对应一个单值的映射。如果你想要一个键映射多个值，那么你就需要将这多个值放到另外的容器中， 比如列表或者集合里面。比如，你可以像下面这样构造这样的字典：

```
d = {
    'a' : [1, 2, 3],
    'b' : [4, 5]
}
e = {
    'a' : {1, 2, 3},
    'b' : {4, 5}
}
```
选择使用列表还是集合取决于你的实际需求。如果你想保持元素的插入顺序就应该使用列表， 如果想去掉重复元素就使用集合（并且不关心元素的顺序问题）。

你可以很方便的使用 collections 模块中的 defaultdict 来构造这样的字典。 defaultdict 的一个特征是它会自动初始化每个 key 刚开始对应的值，所以你只需要关注添加元素操作了。比如：

In [37]:
from collections import defaultdict

d = defaultdict(list)
d['a'].append(1)
d['a'].append(2)
d['b'].append(4)
print(d)
print(d['c']) # 不报错

d = defaultdict(set)
d['a'].add(1)
d['a'].add(2)
d['b'].add(4)
print(d)

defaultdict(<class 'list'>, {'a': [1, 2], 'b': [4]})
[]
defaultdict(<class 'set'>, {'a': {1, 2}, 'b': {4}})


需要注意的是， defaultdict 会自动为将要访问的键（就算目前字典中并不存在这样的键）创建映射实体。 
如果你并不需要这样的特性，你可以在一个普通的字典上使用 setdefault() 方法来代替。比如：

In [ ]:
d = {} # 一个普通的字典
d.setdefault('a', []).append(1)
d.setdefault('a', []).append(2)
d.setdefault('b', []).append(4)
print(d)
print(d['c'])

但是很多程序员觉得 setdefault() 用起来有点别扭。因为每次调用都得创建一个新的初始值的实例（例子程序中的空列表 [] ）。

**讨论**

一般来讲，创建一个多值映射字典是很简单的。但是，如果你选择自己实现的话，那么对于值的初始化可能会有点麻烦， 你可能会像下面这样来实现：

```
d = {}
for key, value in pairs:
    if key not in d:
        d[key] = []
    d[key].append(value)
```

如果使用 defaultdict 的话代码就更加简洁了：

```
d = defaultdict(list)
for key, value in pairs:
    d[key].append(value)
```

### 字典排序

**问题**

你想创建一个字典，并且在迭代或序列化这个字典的时候能够控制元素的顺序。

**解决方案**

为了能控制一个字典中元素的顺序，你可以使用 collections 模块中的 OrderedDict 类。 在迭代操作的时候它会保持元素被插入时的顺序，示例如下：

In [42]:
from collections import OrderedDict

d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4
for key in d:
    print(key,d[key])

foo 1
bar 2
spam 3
grok 4


当你想要构建一个将来需要序列化或编码成其他格式的映射的时候， OrderedDict 是非常有用的。 比如，你想精确控制以 JSON 编码后字段的顺序，你可以先使用 OrderedDict 来构建这样的数据：

In [43]:
import json

json.dumps(d)

'{"foo": 1, "bar": 2, "spam": 3, "grok": 4}'

**讨论**

OrderedDict 内部维护着一个根据键插入顺序排序的双向链表。每次当一个新的元素插入进来的时候， 它会被放到链表的尾部。**对于一个已经存在的键的重复赋值不会改变键的顺序。**

需要注意的是，**一个 OrderedDict 的大小是一个普通字典的两倍，因为它内部维护着另外一个链表。** 所以如果你要构建一个需要大量 OrderedDict 实例的数据结构的时候（比如读取 100,000 行 CSV 数据到一个 OrderedDict 列表中去）， 那么你就得仔细权衡一下是否使用 OrderedDict 带来的好处要大过额外内存消耗的影响。

### 字典的运算

**问题**

怎样在数据字典中执行一些计算操作（比如求最小值、最大值、排序等等）？

**解决方案**

考虑下面的股票名和价格映射字典：

为了对字典值执行计算操作，通常需要**使用 zip() 函数先将键和值反转过来**。 比如，下面是查找最小和最大股票价格和股票值的代码：

In [45]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}

min_price = min(zip(prices.values(),prices.keys()))
max_price = max(zip(prices.values(),prices.keys()))
print(min_price,max_price)

(10.75, 'FB') (612.78, 'AAPL')


类似的，可以使用 **zip() 和 sorted()** 函数来排列字典数据：

In [46]:
prices_sorted = sorted(zip(prices.values(), prices.keys()))
print(prices_sorted)

[(10.75, 'FB'), (37.2, 'HPQ'), (45.23, 'ACME'), (205.55, 'IBM'), (612.78, 'AAPL')]


执行这些计算的时候，需要注意的是 **zip() 函数创建的是一个只能访问一次的迭代器**。 比如，下面的代码就会产生错误

In [48]:
prices_and_names = zip(prices.values(), prices.keys())
print(min(prices_and_names))
print(max(prices_and_names))

(10.75, 'FB')


ValueError: max() arg is an empty sequence

**讨论**

如果你在一个字典上执行普通的数学运算，你会发现它们仅仅**作用于键，而不是值**。比如：

In [49]:
print(max(prices))
print(min(prices))

IBM
AAPL


这个结果并不是你想要的，因为你想要在字典的值集合上执行这些计算。 或许你会尝试着使用字典的 values() 方法来解决这个问题：

In [50]:
print(max(prices.values()))
print(min(prices.values()))

612.78
10.75


不幸的是，通常这个结果同样也不是你想要的。 你可能还想要知道对应的键的信息（比如那种股票价格是最低的？）。

你可以在 min() 和 max() 函数中提供 **key 函数参数**来获取最小值或最大值对应的键的信息。

但是，如果还想要得到最小值，你又得执行一次查找操作。

In [53]:
print(max(prices,key=lambda k: prices[k]))
print(min(prices,key=lambda k: prices[k]))

min_value = prices[min(prices, key=lambda k: prices[k])]

print(min_value)

AAPL
FB
10.75


前面的 zip() 函数方案通过将字典”反转”为 (值，键) 元组序列来解决了上述问题。 当比较两个元组的时候，值会先进行比较，然后才是键。 这样的话你就能通过一条简单的语句就能很轻松的实现在字典上的求最值和排序操作了。

需要注意的是在计算操作中使用到了 (值，键) 对。当多个实体拥有相同的值的时候，键会决定返回结果。 比如，在执行 min() 和 max() 操作的时候，如果恰巧最小或最大值有重复的，那么拥有最小或最大键的实体会返回：

In [54]:
prices = { 'AAA' : 45.23, 'ZZZ': 45.23 }
print(min(zip(prices.values(), prices.keys())))
print(max(zip(prices.values(), prices.keys())))

(45.23, 'AAA')
(45.23, 'ZZZ')


### 查找两字典的相同点

**问题**

怎样在两个字典中寻寻找相同点（比如相同的键、相同的值等等）？

**解决方案**

为了寻找两个字典的相同点，可以简单的在两字典的 keys() 或者 items() 方法返回结果上执行集合操作。比如：

In [55]:
a = {
    'x' : 1,
    'y' : 2,
    'z' : 3
}

b = {
    'w' : 10,
    'x' : 11,
    'y' : 2
}

print(a.keys() & b.keys())
print(a.keys() - b.keys())
print(a.items() & b.items())

{'y', 'x'}
{'z'}
{('y', 2)}


这些操作也可以用于修改或者过滤字典元素。 比如，假如你想以现有字典构造一个排除几个指定键的新字典。 下面利用字典推导来实现这样的需求：

In [56]:
c = {key:a[key] for key in a.keys() - {'z', 'w'}}
print(c)

{'y': 2, 'x': 1}


**讨论**

一个字典就是一个键集合与值集合的映射关系。 

字典的**keys()** 方法返回一个展现键集合的键视图对象。 键视图的一个很少被了解的特性就是它们也**支持集合操作**，比如集合并、交、差运算。 所以，如果你想对集合的键执行一些普通的集合操作，可以直接使用键视图对象而不用先将它们转换成一个 set。

字典的**items()**方法返回一个包含 (键，值) 对的元素视图对象。 这个对象同样也**支持集合操作**，并且可以被用来查找两个字典有哪些相同的键值对。

尽管字典的**values() 方法**也是类似，但是它并**不支持这里介绍的集合操作**。 某种程度上是因为值视图不能保证所有的值互不相同，这样会导致某些集合操作会出现问题。 不过，如果你硬要在值上面执行这些集合操作的话，你可以先将值集合转换成 set，然后再执行集合运算就行了。


### 删除序列相同元素并保持顺序

**问题**

怎样在一个序列上面保持元素顺序的同时消除重复的值？

**解决方案**

如果序列上的值都是 hashable 类型，那么可以很简单的利用集合或者生成器来解决这个问题。比如：

In [5]:
def dequpe(items):
    seen =set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)
            
a = [1, 5, 2, 1, 9, 1, 5, 10]
print(list(dequpe(a)))

[1, 5, 2, 9, 10]


这个方法仅仅在序列中元素为 hashable 的时候才管用。 如果你想消除元素不可哈希（比如 dict 类型）的序列中重复元素的话，你需要将上述代码稍微改变一下，就像这样：

In [10]:
def dedupe(items,key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)
            
a = [ {'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}]
print(list(dedupe(a, key=lambda d: (d['x'],d['y']))))
print(list(dedupe(a, key=lambda d: d['x'])))

[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]
[{'x': 1, 'y': 2}, {'x': 2, 'y': 4}]


如果你想基于单个字段、属性或者某个更大的数据结构来消除重复元素，第二种方案同样可以胜任。

**讨论**

如果你仅仅就是想消除重复元素，通常可以简单的构造一个集合。比如：

In [11]:
a = [1, 5, 2, 1, 9, 1, 5, 10]
print(set(a))

{1, 2, 5, 9, 10}


然而，这种方法**不能维护元素的顺序，生成的结果中的元素位置被打乱**。而上面的方法可以避免这种情况。

在本节中我们使用了生成器函数让我们的函数更加通用，不仅仅是局限于列表处理。 比如，如果如果你想读取一个文件，消除重复行，你可以很容易像这样做：

```
with open(somefile,'r') as f:
for line in dedupe(f):
    ...
```
上述key函数参数模仿了 sorted() , min() 和 max() 等内置函数的相似功能。

### 命名切片

计算机科学中，只有硬编码（hardcode），以及非硬编码，有人也成为“软编码”。 硬编码和软编码的区别是：**软编码可以在运行时确定，修改；而硬编码是不能够改变的。**

**问题**

如果你的程序包含了大量无法直视的硬编码切片，并且你想清理一下代码。

**解决方案**

假定你要从一个记录（比如文件或其他类似格式）中的某些固定位置提取字段：

In [13]:
######    0123456789012345678901234567890123456789012345678901234567890'
record = '....................100 .......513.25 ..........'
cost = int(record[20:23]) * float(record[31:37])
print(cost)

51325.0


与其那样写，为什么不想这样命名切片呢：

In [14]:
SHARES = slice(20, 23)
PRICE = slice(31, 37)
cost = int(record[SHARES]) * float(record[PRICE])
print(cost)

51325.0


在这个版本中，你避免了使用大量难以理解的硬编码下标。这使得你的代码更加清晰可读。

**讨论**

一般来讲，代码中如果出现大量的硬编码下标会使得代码的可读性和可维护性大大降低。 比如，如果你回过来看看一年前你写的代码，你会摸着脑袋想那时候自己到底想干嘛啊。 这是一个很简单的解决方案，它让你更加清晰的表达代码的目的。

内置的 slice() 函数创建了一个切片对象。所有使用切片的地方都可以使用切片对象。比如:

In [16]:
items = [0, 1, 2, 3, 4, 5, 6]
a = slice(2,4)
print(items[2:4])
print(items[a])
items[a] = [10,11]
print(items)
del items[a]
print(items)

[2, 3]
[2, 3]
[0, 1, 10, 11, 4, 5, 6]
[0, 1, 4, 5, 6]


如果你有一个切片对象a，你可以分别调用它的 a.start , a.stop , a.step 属性来获取更多的信息。比如：

In [17]:
a = slice(5,50,2)
print(a.start)
print(a.stop)
print(a.step)

5
50
2


另外，你还可以通过调用切片的 indices(size) 方法将它映射到一个已知大小的序列上。 这个方法返回一个三元组 (start, stop, step) ，所有的值都会被缩小，直到适合这个已知序列的边界为止。 这样，使用的时就不会出现 IndexError 异常。比如：

In [19]:
s = "HelloWorld"
print(a.indices(len(s)))
for i in range(*a.indices(len(s))):
    print(s[i])

(5, 10, 2)
W
r
d


### 序列中出现次数最多的元素

**问题**

怎样找出一个序列中出现次数最多的元素呢？

**解决方案**

collections.Counter 类就是专门为这类问题而设计的， 它甚至有一个有用的 most_common() 方法直接给了你答案。

为了演示，先假设你有一个单词列表并且想找出哪个单词出现频率最高。你可以这样做：

In [21]:
words = [
    'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
    'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the',
    'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
    'my', 'eyes', "you're", 'under'
]

from collections import Counter
word_counts = Counter(words)
top_three = word_counts.most_common(3)
print(top_three)

[('eyes', 8), ('the', 5), ('look', 4)]


**讨论**

作为输入， Counter 对象可以接受任意的由可哈希（hashable）元素构成的序列对象。 在底层实现上，一个 Counter 对象就是一个字典，将元素映射到它出现的次数上。比如：

In [22]:
print(word_counts['not'])
print(word_counts['eyes'])

1
8


如果你想手动增加计数，可以简单的用加法，或者你可以使用 update() 方法：

In [25]:
morewords = ['why','are','you','not','looking','in','my','eyes']
for word in morewords:
    word_counts[word] += 1
print(word_counts['eyes'])

word_counts.update(morewords)
print(word_counts['eyes'])

10
11


Counter 实例一个鲜为人知的特性是它们可以很容易的跟数学运算操作相结合。比如

In [27]:
a = Counter(words)
b = Counter(morewords)
print(a)
print(b)
c = a+b
print(c)
d = a-b
print(d)

Counter({'eyes': 8, 'the': 5, 'look': 4, 'into': 3, 'my': 3, 'around': 2, 'not': 1, "don't": 1, "you're": 1, 'under': 1})
Counter({'why': 1, 'are': 1, 'you': 1, 'not': 1, 'looking': 1, 'in': 1, 'my': 1, 'eyes': 1})
Counter({'eyes': 9, 'the': 5, 'look': 4, 'my': 4, 'into': 3, 'not': 2, 'around': 2, "don't": 1, "you're": 1, 'under': 1, 'why': 1, 'are': 1, 'you': 1, 'looking': 1, 'in': 1})
Counter({'eyes': 7, 'the': 5, 'look': 4, 'into': 3, 'my': 2, 'around': 2, "don't": 1, "you're": 1, 'under': 1})


毫无疑问， Counter 对象在几乎所有需要制表或者计数数据的场合是非常有用的工具。 在解决这类问题的时候你应该优先选择它，而不是手动的利用字典去实现。

### 通过某个关键字排序一个字典列表

**问题**

你有一个字典列表，你想根据某个或某几个字典字段来排序这个列表。

**解决方案**

通过使用 operator 模块的 itemgetter 函数，可以非常容易的排序这样的数据结构。 假设你从数据库中检索出来网站会员信息列表，并且以下列的数据结构返回：

In [28]:
rows = [
    {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
    {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
    {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
    {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}
]

from operator import itemgetter
rows_by_fname = sorted(rows,key=itemgetter('fname'))
rows_by_uid = sorted(rows,key=itemgetter('uid'))
print(rows_by_fname)
print(rows_by_uid)

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]
[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]


itemgetter() 函数也支持多个 keys，比如下面的代码

In [29]:
rows_by_lfname = sorted(rows, key=itemgetter('lname','fname'))
print(rows_by_lfname)

[{'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}]
